In [4]:
# Install Pandas and Requests
# pip install pandas requests // Execute in Terminal

# Import the required libraries
import requests
import pandas as pd


# # Define the URL for the JSON file
# url_json = "https://climateknowledgeportal.worldbank.org/themes/cckp/data/geonames.json"

# # Fetch the JSON data using a GET request
# response_json = requests.get(url_json)

# # Check if the request was successful
# if response_json.status_code == 200:
#     # Parse the JSON response into a Python dictionary
#     data_json = response_json.json()
#     print("Data fetched successfully!")
# else:
#     print(f"Failed to fetch data. Status code: {response.status_code}")
#     data_json = None

# # Convert the JSON data into a tabular format (DataFrame) for readability
# if data_json:
#     # Flatten the nested structure into a list of dictionaries
#     flat_data_json = []
#     for country_code, details in data_json["country"].items():
#         country_name = details["N"]
#         for subnational_code, subnational_name in details["S"].items():
#             flat_data_json.append({
#                 "Country Code": country_code,
#                 "Country Name": country_name,
#                 "Subnational Code": subnational_code,
#                 "Subnational Name": subnational_name
#             })

#     # Convert the list into a pandas DataFrame
#     df_json = pd.DataFrame(flat_data_json)
    
#     # Display the first few rows of the DataFrame
#     print(df_json.head())

#     # Save to a CSV file (optional)
#     df_json.to_csv("worldbank_geonames.csv", index=False)
#     print("Data saved to worldbank_geonames.csv")

def fetch_and_process_geonames_data():
    """
    Fetches JSON geonames data from the World Bank's endpoint,
    processes it into a pandas DataFrame, and optionally saves it to a CSV file.
    """
    # Define the URL for the geonames JSON file
    url_geonames_json = "https://climateknowledgeportal.worldbank.org/themes/cckp/data/geonames.json"

    try:
        # Fetch the JSON data using a GET request
        response_geonames_json = requests.get(url_geonames_json)
        response_geonames_json.raise_for_status()  # Raise an error for bad HTTP responses

        # Parse the JSON response into a Python dictionary
        geonames_json = response_geonames_json.json()
        print("Geonames data fetched successfully as JSON-file!")

        # Flatten the nested structure into a list of dictionaries
        flat_geonames_data_json = [
            {
                "Country Code": country_code,
                "Country Name": details["N"],
                "Subnational Code": subnational_code,
                "Subnational Name": subnational_name
            }
            for country_code, details in geonames_json["country"].items()
            for subnational_code, subnational_name in details["S"].items()
        ]

        # Convert the list into a pandas DataFrame
        geonames_df = pd.DataFrame(flat_geonames_data_json)
        print("Geonames data flattened succesfully to a DataFrame!")

        # Save to a CSV file (optional)
        geonames_csv = "geonames_csv.csv"
        geonames_df.to_csv(geonames_csv, index=False)
        print(f"Geonames data saved to {geonames_csv}")

        # Return the DataFrame for further use
        return geonames_df

    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch geonames data: {e}")
        return None
    
# Execute the function and fetch the geonames data
geonames_df = fetch_and_process_geonames_data()

# Display the first few rows (if the data was fetched successfully)
if geonames_df is not None:
    print(geonames_df.head())

Geonames data fetched successfully as JSON-file!
Geonames data flattened succesfully to a DataFrame!
Geonames data saved to geonames_csv.csv
  Country Code Country Name Subnational Code Subnational Name
0          AFG  Afghanistan           AFG.11       Badakhshan
1          AFG  Afghanistan           AFG.12          Badghis
2          AFG  Afghanistan           AFG.13          Baghlan
3          AFG  Afghanistan           AFG.14            Balkh
4          AFG  Afghanistan           AFG.15           Bamyan
